In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from scipy.stats import chisquare


import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline

In [3]:
df = pd.read_csv('/Users/dspitzfaden/Galvanize/capstone/data_files/yelp_reviews_with_period.csv')
df= df[df['inspection_grade']!='X']
df['A'] = np.where(df['inspection_grade']== 'A', 1, 0) #1 is A, 0 is B or C
df['B'] = np.where(df['inspection_grade']== 'B', 1, 0)
df['C'] = np.where(df['inspection_grade']== 'C', 1, 0)

grade_df= df[['yelp_business_id', 'inspection_date', 'last_inspection','inspection_grade']].reset_index().sort_values(['yelp_business_id', 'inspection_date', 'last_inspection']).reset_index()
grade_df= grade_df.drop_duplicates(['yelp_business_id', 'inspection_date', 'last_inspection'])
grade_df['A'] = np.where(grade_df['inspection_grade']== 'A', 1, 0) #1 is A, 0 is B or C


grouped_text= df.groupby(['yelp_business_id', 'inspection_date', 'last_inspection'])['yelp_review_text'].apply(lambda x: "{%s}" % '~~ '.join(x)).reset_index().sort_values(['yelp_business_id', 'inspection_date', 'last_inspection'])

In [3]:
count_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=10000, stop_words='english', ngram_range=(2,2))
tfidf = count_vectorizer.fit_transform(grouped_text['yelp_review_text'].values)
features= count_vectorizer.vocabulary_

In [4]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, grade_df['A'].values,\
                                                    test_size=0.2)

In [5]:
rf = RandomForestClassifier(class_weight="balanced", max_depth=6).fit(X_train, y_train)
rf_pred= rf.predict(X_test)
rf_tn, rf_fp, rf_fn, rf_tp = confusion_matrix(rf_pred, y_test).ravel()
precision, recall = (rf_tp / (rf_tp+rf_fp)), (rf_tp/(rf_tp+rf_fn))
print('RF Precision: {}   RF Recall: {}'.format(precision, recall))

RF Precision: 0.7995933778681382   RF Recall: 0.8714783159227604


In [13]:
chi2_nlp = chi2(tfidf, grade_df['inspection_grade'].values)

In [14]:
chi_vocab_score = list(chi2_nlp[0])
p_val = list(chi2_nlp[1])

In [21]:
vocab_chi = zip(features, chi_vocab_score)
vocab_p_val = zip(features, p_val)

sorted_chi= sorted(vocab_chi, key=lambda x: x[1], reverse= True)
sorted_p_val = sorted(vocab_p_val, key=lambda x: x[1], reverse= False)

In [16]:
ac_chisq, ac_p = chisquare(ccounts, acounts)

In [17]:
ac_chisq

0.022424034482787285

In [11]:
bc_chisq, bc_p = chisquare(ccounts, bcounts)

In [12]:
ba_chisq, ba_p = chisquare(acounts, bcounts)

In [13]:
bc_p

0.99994480134810815

In [14]:
ba_p

0.99999932468375019